In [ ]:
# Ali Ahmad and David Nguyen, TSG Lab 


In [1]:
# Load the dependencies

import cv2
import numpy as np
import os


In [5]:
# Set working directory by changing the file path in the quotation marks

os.chdir('/Users/davidnguyen/Downloads/temp')
path = os.getcwd()
# print(path)
contents = os.listdir(path)
contents
hidden_files = []

# *this is the weird for-loop referenced above
for item in contents:
    if item.startswith(".") == True:
        hidden_files.append(item)

contents


['Test Closed Shape_long-1.png', 'Test Closed Shape_long-2.png']

In [7]:
# Load these functions

def remove_hidden(thing):
    unwanted = thing
    contents.remove(unwanted)

for item in hidden_files:
    remove_hidden(item)

def save_image(filename,image):
    
    split_string = filename.split(".", 1)
    stripped_name = split_string[0]

    # append suffix to the stripped file name
    final_name = f'{stripped_name}_rot.jpg'
    cv2.imwrite(final_name, image)

In [9]:
# Run this main loop


for filename in contents:
    
    
    # Load the image
    image = cv2.imread(os.path.join(path,filename))
    image_copy = image.copy()
    origimage=image

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray, 100, 200)

    # Find contours in the edges
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Sort the contours by their area (largest to smallest)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)


    # Get the largest contour (irregular shape)
    irregular_contour = contours[0]

    # Calculate the maximum distance between two points of the irregular shape contour
    max_distance = 0
    point_1 = None
    point_2 = None

    for i in range(len(irregular_contour)):
        for j in range(i + 1, len(irregular_contour)):
            distance = np.linalg.norm(irregular_contour[i][0] - irregular_contour[j][0])
            if distance > max_distance:
                max_distance = distance
                point_1 = tuple(irregular_contour[i][0])
                point_2 = tuple(irregular_contour[j][0])

    # Draw the longest line on the original image
    cv2.line(image, point_1, point_2, (0, 0, 255), 2)

    # Create a black image for drawing the contour
    contour_image = np.zeros_like(image)

    # Draw the contour on the separate image
    cv2.drawContours(contour_image, [irregular_contour], 0, (0, 255, 0), 2)

    # Calculate the rotation angle to make the line horizontal
    angle = np.arctan2(point_2[1] - point_1[1], point_2[0] - point_1[0]) * 180 / np.pi

    # Rotate the original image to make the line horizontal
    rows, cols, _ = image_copy.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols // 2, rows // 2), angle, 1)
    rotated_image = cv2.warpAffine(image_copy, rotation_matrix, (cols, rows), borderValue=(255, 255, 255))

    # Remove the drawn line from the rotated image
    rotated_image_no_line = rotated_image.copy()
    #cv2.line(rotated_image_no_line, point_1, point_2, (0, 0, 0), 2)

    save_image(filename,rotated_image_no_line)
    
def check_rotation_correctness(filename, rotated_image):
    if "_rot" in filename:
        # Convert the image to grayscale
        gray = cv2.cvtColor(rotated_image, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection
        edges = cv2.Canny(gray, 100, 200)

        # Find contours in the edges
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        # Sort the contours by their area (largest to smallest)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)

        # Get the largest contour (irregular shape)
        irregular_contour = contours[0]

        # Calculate the orientation of the longest line in the contour
        _, _, angle = cv2.fitLine(irregular_contour, cv2.DIST_L2, 0, 0.01, 0.01)

        # If the line is not approximately horizontal within a tolerance, consider rotation incorrect
        if abs(angle) > 5:  # You can adjust this tolerance level as needed
            return True  # Rotation incorrect
    return False  # Rotation correct


incorrectly_rotated_images = []

for filename in contents:
    # Load the image
    image = cv2.imread(os.path.join(path, filename))
    image_copy = image.copy()

    # Your existing rotation and saving code here...

    # Check if the rotation is correct for "_rot" files
    if check_rotation_correctness(filename, rotated_image_no_line):
        incorrectly_rotated_images.append(filename)

print("Incorrectly rotated images:", incorrectly_rotated_images)

Incorrectly rotated images: []
